# ELephantSQL Database Builder:

A script for importing local aviation data; modifying it to create various tables of interest; and then importing those tables into a database so that my web app can access it

By Kenneth Burchfiel

Released under the MIT License

In [1]:
import json
import sqlalchemy
import pandas as pd



Connecting to ElephantSQL database:

In [2]:
with open ("..\key_paths\path_to_keys_folder.txt") as file:
    key_path = file.read()

In [3]:
with open(key_path+"\\elephantsql_kjb3webchartapp_db_url.txt") as file:
    db_url = file.read()
    # This code reads in my database's URL. This URL is listed on the home page for my database within elephantsql.com. As shown below, SQLAlchemy can use this URL to connect to the database. 

elephantsql_db_url_for_sqlalchemy = db_url.replace('postgres://', 'postgresql://')
# This change, which is required for SQLAlchemy to work correctly, is based on the code suggested at:
# # https://help.heroku.com/ZKNTJQSK/why-is-sqlalchemy-1-4-x-not-connecting-to-heroku-postgres

In [4]:
elephantsql_engine = sqlalchemy.create_engine(elephantsql_db_url_for_sqlalchemy)

## Loading in data

Next, I'll load in a .csv file that was created using the [flights_table_builder](https://github.com/kburchfiel/route_maps_builder/blob/master/flights_table_builder.ipynb) script within my [Route Maps Builder](https://github.com/kburchfiel/route_maps_builder) program. The original file is too large to host on GitHub, but it can be found within [this Google Drive page](https://drive.google.com/file/d/1gUozlxpVIO-gQ6d-AQ4Qgpg91Ut3kqUD/view?usp=sharing).



In [5]:
pd.set_option('display.max_columns', 100)

In [6]:
df_flights = pd.read_csv(r'C:\Users\kburc\D1V1\Documents\!Dell64docs\Programming\py\kjb3_programs\route_maps_builder\routes_planes_coordinates_for_mapping_2018.csv')
df_flights.drop('Unnamed: 0', axis = 1,inplace=True)
df_flights['UNIQUE_CARRIER_NAME'].fillna("Unknown", inplace = True) # This line will allow rows without an airline name to still be processed by pivot table operations that reference those names.
df_flights

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.980,-95.340,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,21502.0,76.0,0.0,0.0,0.0,476.0,157.0,138.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,IAD,38.944,-77.456,Domestic,Domestic,Domestic,IAD_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349168,1146.0,573.0,1031400.0,5157.0,3693.0,0.0,0.0,91.0,26816.0,20972.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,9,1,F,DU,NaN,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
349169,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket,

## Pivoting and filtering data

This table currently contains around 350,000 rows and 65 columns. However, my web app doesn't need all of those rows present in its own PostgreSQL database; instead, it just needs enough data to generate various charts. Therefore, I'll now create some modified versions of this data and then upload it to the PostgreSQL database.

In [7]:
df_airline_origin_dest_plane_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER', 'ORIGIN_DEST', 'Plane_Type_Text'], values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False)
df_airline_origin_dest_plane_pivot = df_airline_origin_dest_plane_pivot.query("PASSENGERS > 100000").copy()
df_airline_origin_dest_plane_pivot['Airport 1'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[0]
df_airline_origin_dest_plane_pivot['Airport 2'] = df_airline_origin_dest_plane_pivot['ORIGIN_DEST'].str.split('_').str[1]
df_airline_origin_dest_plane_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers","ORIGIN_DEST":"Origin_Dest"},inplace=True)
df_airline_origin_dest_plane_pivot.reset_index(drop=True,inplace=True)
df_airline_origin_dest_plane_pivot

,Airline,Origin_Dest,Plane_Type_Text,Passengers,Airport 1,Airport 2
0,HA,HNL_OGG,Boeing 717-200,1954139.0,HNL,OGG
1,DL,ATL_MCO,Boeing 757-200,1417832.0,ATL,MCO
2,HA,HNL_KOA,Boeing 717-200,1281221.0,HNL,KOA
3,WN,DAL_HOU,Boeing 737-700/700LR/Max 7,1260362.0,DAL,HOU
4,AA,DFW_LAX,Airbus Industrie A321/Lr,1257512.0,DFW,LAX
...,...,...,...,...,...,...
2799,NK,FLL_LGA,Airbus Industrie A320-100/200,100381.0,FLL,LGA
2800,EI,DUB_MCO,Airbus Industrie A330-200,100329.0,DUB,MCO
2801,YX,DCA_MCI,Embraer ERJ-175,100303.0,DCA,MCI
2802,WN,BNA_CLT,Boeing 737-700/700LR/Max 7,100282.0,BNA,CLT


The following cell exports this pivot table to .csv format locally (so that it can be accessed without a database connection) and to .sql format online.

In [8]:
df_airline_origin_dest_plane_pivot.to_csv('local_copy_of_airports_airlines_aircraft_2018.csv', index = False)

df_airline_origin_dest_plane_pivot.to_sql("airports_airlines_aircraft_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

804

I believe the output of the above cell shows the smallest 3 digits of the number of rows updated within the table. Thus, if 2,804 rows were created within the table, the output will be 804.

The following cell reads the new database table back into Pandas in order to verify that it was imported successfully:

In [9]:
pd.read_sql("airports_airlines_aircraft_2018", con = elephantsql_engine)

,Airline,Origin_Dest,Plane_Type_Text,Passengers,Airport 1,Airport 2
0,HA,HNL_OGG,Boeing 717-200,1954139.0,HNL,OGG
1,DL,ATL_MCO,Boeing 757-200,1417832.0,ATL,MCO
2,HA,HNL_KOA,Boeing 717-200,1281221.0,HNL,KOA
3,WN,DAL_HOU,Boeing 737-700/700LR/Max 7,1260362.0,DAL,HOU
4,AA,DFW_LAX,Airbus Industrie A321/Lr,1257512.0,DFW,LAX
...,...,...,...,...,...,...
2799,NK,FLL_LGA,Airbus Industrie A320-100/200,100381.0,FLL,LGA
2800,EI,DUB_MCO,Airbus Industrie A330-200,100329.0,DUB,MCO
2801,YX,DCA_MCI,Embraer ERJ-175,100303.0,DCA,MCI
2802,WN,BNA_CLT,Boeing 737-700/700LR/Max 7,100282.0,BNA,CLT


## Creating additional tables for the database:

The following cells create filtered versions of df_flights that show only domestic or international destinations. This allows me to ensure, for instance, that a list of the top 20 US airports by international traffic won't contain non-US airports.

In [10]:
df_flights_domestic_destination_airports = df_flights.query("Destination_Region == 'Domestic'")
df_flights_domestic_destination_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.980,-95.340,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,21502.0,76.0,0.0,0.0,0.0,476.0,157.0,138.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,IAD,38.944,-77.456,Domestic,Domestic,Domestic,IAD_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349168,1146.0,573.0,1031400.0,5157.0,3693.0,0.0,0.0,91.0,26816.0,20972.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,3,9,1,F,DU,NaN,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
349169,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket,

In [11]:
df_flights_intl_destination_airports = df_flights.query("Destination_Region == 'International'")
df_flights_intl_destination_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
33,0.0,1.0,21502.0,76.0,66.0,0.0,0.0,182.0,43.0,32.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,14576,1457606,34576,ROC,"Rochester, NY",NY,36.0,New York,US,United States,22,16149,1614904,36149,YOW,"Ottawa, Canada",ON,NaN,Ontario,CA,Canada,936,6,638,1,2018,1,2,1,F,IU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ROC,43.119,-77.672,YOW,45.323,-75.669,Domestic,International,International,ROC_YOW
89,0.0,1.0,21405.0,0.0,0.0,0.0,0.0,1335.0,232.0,220.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,US,United States,82,11874,1187406,31874,GDL,"Guadalajara, Mexico",NaN,NaN,NaN,MX,Mexico,148,6,635,2,2018,1,2,3,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,DEN,39.858,-104.667,GDL,20.522,-103.311,Domestic,International,International,DEN_GDL
90,0.0,1.0,21323.0,0.0,0.0,5090.0,0.0,165.0,47.0,33.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,14886,1488605,34886,SLW,"Saltillo, Mexico",NaN,NaN,NaN,MX,Mexico,148,6,635,2,2018,1,2,1,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,LRD,27.544,-99.461,SLW,25.549,-100.929,Domestic,International,International,LRD_SLW
91,0.0,2.0,42143.0,0.0,0.0,0.0,0.0,565.0,200.0,170.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,15246,1524605,33252,TLC,"Toluca, Mexico",NaN,NaN,NaN,MX,Mexico,148,6,635,2,2018,1,2,2,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,LRD,27.544,-99.461,TLC,19.337,-99.566,Domestic,International,International,LRD_TLC
92,0.0,1.0,20995.0,0.0,0.0,4069.0,0.0,878.0,148.0,127.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13244,1324402,33244,MEM,"Memphis, TN",TN,47.0,Tennessee,US,United States,54,13514,1351403,33514,MTY,"Monterrey, Mexico",NaN,NaN,NaN,MX,Mexico,148,6,635,2,2018,1,2,2,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,MEM,35.042,-89.977,MTY,25.778,-100.107,Domestic,International,International,MEM_MTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347746,246.0,123.0,221400.0,1107.0,455.0,0.0,0.0,96.0,5904.0,4797.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,14843,1484306,34819,SJU,"San Juan, PR",PR,72.0,Puerto Rico,US,United States,3,11519,1151907,34448,EIS,"Road Town, British Virgin Islands",NaN,NaN,NaN,VG,British Virgin Islands,282,1,125,1,2018,3,8,1,F,IU,NaN,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,SJU,18.439,-66.002,EIS,18.445,-64.543,Domestic,International,International,EIS_SJU
348082,262.0,131.0,235800.0,1179.0,761.0,0.0,0.0,96.0,6288.0,5109.0,9K,20253.0,Cape Air,070

In [12]:
df_flights_domestic_destination_airports_intl_traffic = df_flights.query("Destination_Region == 'Domestic' & Route_Type == 'International'")
df_flights_domestic_destination_airports_intl_traffic

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
36,0.0,1.0,21502.0,76.0,75.0,0.0,0.0,803.0,256.0,203.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,16217,1621702,36083,YUL,"Montreal, Canada",QC,NaN,Quebec,CA,Canada,941,11057,1105703,31057,CLT,"Charlotte, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,2,F,IU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,YUL,45.468,-73.741,CLT,35.214,-80.943,International,Domestic,International,CLT_YUL
88,0.0,1.0,21323.0,0.0,0.0,0.0,0.0,468.0,75.0,60.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,10638,1063802,30638,BJX,"Leon/Guanajuato, Mexico",NaN,NaN,NaN,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,BJX,20.993,-101.481,LRD,27.544,-99.461,International,Domestic,International,BJX_LRD
93,0.0,2.0,42400.0,0.0,0.0,0.0,0.0,128.0,82.0,54.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,13514,1351403,33514,MTY,"Monterrey, Mexico",NaN,NaN,NaN,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,MTY,25.778,-100.107,LRD,27.544,-99.461,International,Domestic,International,LRD_MTY
97,0.0,1.0,21148.0,0.0,0.0,0.0,0.0,376.0,80.0,60.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,14879,1487901,34879,SLP,"San Luis Potosi, Mexico",NaN,NaN,NaN,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,SLP,22.254,-100.931,LRD,27.544,-99.461,International,Domestic,International,LRD_SLP
98,0.0,3.0,63313.0,0.0,0.0,0.0,0.0,165.0,141.0,87.0,AMQ,20201.0,Ameristar Air Cargo,16057,I,AMQ,Ameristar Air Cargo,1,4.0,14886,1488605,34886,SLW,"Saltillo, Mexico",NaN,NaN,NaN,MX,Mexico,148,13061,1306106,33038,LRD,"Laredo, TX",TX,48.0,Texas,US,United States,74,6,635,2,2018,1,2,1,P,IU,NaN,635,McDonnell Douglas DC-9-15F,6,"Jet, 2-Engine",2,Freight Configuration,SLW,25.549,-100.929,LRD,27.544,-99.461,International,Domestic,International,LRD_SLW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347799,248.0,124.0,223200.0,1116.0,663.0,0.0,0.0,96.0,5952.0,4836.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,11519,1151907,34448,EIS,"Road Town, British Virgin Islands",NaN,NaN,NaN,VG,British Virgin Islands,282,14843,1484306,34819,SJU,"San Juan, PR",PR,72.0,Puerto Rico,US,United States,3,1,125,1,2018,3,8,1,F,IU,NaN,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,EIS,18.445,-64.543,SJU,18.439,-66.002,International,Domestic,International,EIS_SJU
348078,262.0,131.0,235800.0,1179.0,695.0,0.0,0.0,96.0,6288.0,5109.0,9K,20253.0,Cape Air,

# Creating a U.S. airports table

I'll now create a list of the top 20 U.S. airports by arriving passenger traffic that will also include (1) domestic and international traffic breakdowns and (2) each airport's rank. (I chose to focus on arriving traffic rather than departing traffic, but either option can work, as long as you're consistent in that decision throughout your code.) First, I'll calculate the ranks of the top 20 U.S. airports by arriving passenger traffic:

In [13]:
df_top_20_airport_ranks = df_flights_domestic_destination_airports.pivot_table(index = "DEST", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).head(20).reset_index()
df_top_20_airport_ranks['Rank'] = df_top_20_airport_ranks['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airport_ranks

,DEST,PASSENGERS,Rank
0,ATL,47388875.0,1
1,LAX,41502828.0,2
2,ORD,36572207.0,3
3,DFW,30823751.0,4
4,JFK,29744941.0,5
5,DEN,29527533.0,6
6,SFO,26974833.0,7
7,LAS,23511649.0,8
8,MCO,23178413.0,9
9,SEA,22558466.0,10


Next, I'll create a list of these top 20 airports:

In [14]:
top_20_airports = df_top_20_airport_ranks['DEST'].to_list()
top_20_airports

['ATL',
 'LAX',
 'ORD',
 'DFW',
 'JFK',
 'DEN',
 'SFO',
 'LAS',
 'MCO',
 'SEA',
 'EWR',
 'PHX',
 'IAH',
 'MIA',
 'CLT',
 'BOS',
 'FLL',
 'MSP',
 'DTW',
 'PHL']

I'll then use this list to create a filtered version of df_flights:

In [15]:
df_top_20_domestic_airports = df_flights.query("DEST in @top_20_airports")
df_top_20_domestic_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779,Domestic,Domestic,Domestic,IAD_JFK
11,0.0,2.0,43004.0,152.0,114.0,0.0,0.0,762.0,322.0,264.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12953,1295304,31703,LGA,"New York, NY",NY,36.0,New York,US,United States,22,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,LGA,40.777,-73.872,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_LGA
14,0.0,1.0,21502.0,76.0,67.0,0.0,0.0,304.0,80.0,57.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12992,1299206,32600,LIT,"Little Rock, AR",AR,5.0,Arkansas,US,United States,71,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,LIT,34.729,-92.224,DFW,32.896,-97.037,Domestic,Domestic,Domestic,DFW_LIT
15,0.0,2.0,43004.0,152.0,73.0,0.0,0.0,404.0,197.0,129.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,13204,1320402,31454,MCO,"Orlando, FL",FL,12.0,Florida,US,United States,33,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,1,L,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,MCO,28.429,-81.316,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_MCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349161,910.0,455.0,819000.0,4095.0,2360.0,0.0,0.0,91.0,21294.0,16653.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,Massachusetts,US,United States,13,10721,1072102,30721,BOS,"Boston, MA",MA,25.0,Massachusetts,US,United States,13,1,125,1,2018,2,5,1,F,DU,NaN,125,Cessna C-402/402A/402B,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005,Domestic,Domestic,Domestic,ACK_BOS
349165,1096.0,548.0,986400.0,4932.0,2913.0,0.0,0.0,91.0,25646.0,20057.0,9K,20253.0,Cape Air,07021,D,9K,Cape Air,1,6.0,10154,1015405,30154,ACK,"Nantucket, MA",MA,25.0,M

In [16]:
df_top_20_domestic_airports_pivot = df_top_20_domestic_airports.pivot_table(index = ['DEST', 'Route_Type'], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).reset_index()
df_top_20_domestic_airports_pivot = df_top_20_domestic_airports_pivot.merge(df_top_20_airport_ranks[['DEST', 'Rank']], on = 'DEST')
df_top_20_domestic_airports_pivot.sort_values(['Rank', 'Route_Type'], inplace = True)
df_top_20_domestic_airports_pivot.rename(columns={"DEST":"Airport","PASSENGERS":"Passengers"}, inplace = True)
df_top_20_domestic_airports_pivot

,Airport,Route_Type,Passengers,Rank
0,ATL,Domestic,41606158.0,1
1,ATL,International,5782717.0,1
4,LAX,Domestic,29643615.0,2
5,LAX,International,11859213.0,2
2,ORD,Domestic,30105889.0,3
3,ORD,International,6466318.0,3
8,DFW,Domestic,26907771.0,4
9,DFW,International,3915980.0,4
27,JFK,Domestic,14100771.0,5
26,JFK,International,15644170.0,5


I'm also interested in seeing which U.S. airports had the highest international passenger traffic, so I'll create that list as well:

In [17]:
df_top_20_airports_intl_traffic = df_flights_domestic_destination_airports_intl_traffic.pivot_table(index = 'destination_iata_code', values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).head(20) # Calculates the top 20 airports by arriving passenger count.
df_top_20_airports_intl_traffic.reset_index(inplace=True)
df_top_20_airports_intl_traffic['Rank'] = df_top_20_airports_intl_traffic['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airports_intl_traffic.rename(columns={"destination_iata_code":"Airport","PASSENGERS":"Passengers"},inplace=True)
df_top_20_airports_intl_traffic

,Airport,Passengers,Rank
0,JFK,15644170.0,1
1,LAX,11859213.0,2
2,MIA,10112517.0,3
3,EWR,6555068.0,4
4,SFO,6553134.0,5
5,ORD,6466318.0,6
6,ATL,5782717.0,7
7,IAH,5151668.0,8
8,FLL,4266110.0,9
9,DFW,3915980.0,10


Now that I've created these tables, I'll go ahead and export them to both a local .csv file and to my SQL database:

In [18]:
df_top_20_airports_intl_traffic.to_csv('top_20_airports_by_intl_pax_arrivals_2018.csv', index = False)

df_top_20_airports_intl_traffic.to_sql("top_20_airports_by_intl_pax_arrivals_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

20

In [19]:
df_top_20_domestic_airports_pivot.to_csv('top_20_airports_by_pax_arrivals_2018.csv', index = False)

df_top_20_domestic_airports_pivot.to_sql("top_20_airports_by_pax_arrivals_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

40

# Creating an airlines table

I'll now go through a similar process to create a list of the top 20 airlines by US airport arrivals that distinguishes between domestic and international traffic.

In [20]:
df_top_20_airline_ranks = df_flights.pivot_table(index = 'UNIQUE_CARRIER', values = 'PASSENGERS', aggfunc = 'sum').reset_index().sort_values('PASSENGERS', ascending = False).head(20).reset_index(drop=True)
df_top_20_airline_ranks['Rank'] = df_top_20_airline_ranks['PASSENGERS'].rank(ascending = False).astype(int) # See https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html
df_top_20_airline_ranks

,UNIQUE_CARRIER,PASSENGERS,Rank
0,WN,165139267.0,1
1,AA,146267939.0,2
2,DL,143049045.0,3
3,UA,110385926.0,4
4,B6,41635897.0,5
5,AS,33755225.0,6
6,NK,28777722.0,7
7,OO,26810281.0,8
8,F9,19411454.0,9
9,YX,17474019.0,10


In [21]:
df_top_20_airline_list = df_top_20_airline_ranks['UNIQUE_CARRIER'].to_list()
df_top_20_airline_list

['WN',
 'AA',
 'DL',
 'UA',
 'B6',
 'AS',
 'NK',
 'OO',
 'F9',
 'YX',
 'YV',
 '9E',
 'HA',
 'MQ',
 'OH',
 'BA',
 'EV',
 'LH',
 'CP',
 'WS']

In [22]:
df_top_20_airlines = df_flights.query("UNIQUE_CARRIER in @df_top_20_airline_list")
df_top_20_airlines

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12.0,Florida,US,United States,33,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153,Domestic,Domestic,Domestic,FLL_IAD
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,12478,1247805,31703,JFK,"New York, NY",NY,36.0,New York,US,United States,22,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779,Domestic,Domestic,Domestic,IAD_JFK
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12266,1226603,31453,IAH,"Houston, TX",TX,48.0,Texas,US,United States,74,14685,1468502,34685,SAV,"Savannah, GA",GA,13.0,Georgia,US,United States,34,6,638,1,2018,1,2,2,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.980,-95.340,SAV,32.127,-81.202,Domestic,Domestic,Domestic,IAH_SAV
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12323,1232305,32323,ILM,"Wilmington, NC",NC,37.0,North Carolina,US,United States,36,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37.0,North Carolina,US,United States,36,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787,Domestic,Domestic,Domestic,ILM_RDU
4,0.0,1.0,21502.0,76.0,0.0,0.0,0.0,476.0,157.0,138.0,9E,20363.0,Endeavor Air Inc.,07024,D,9E,Endeavor Air Inc.,2,2.0,12339,1233904,32337,IND,"Indianapolis, IN",IN,18.0,Indiana,US,United States,42,12264,1226402,30852,IAD,"Washington, DC",VA,51.0,Virginia,US,United States,38,6,638,1,2018,1,2,1,F,DU,NaN,638,Canadair CRJ 900,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,IAD,38.944,-77.456,Domestic,Domestic,Domestic,IAD_IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349158,855.0,854.0,27027900.0,109312.0,81674.0,172551.0,0.0,100.0,33015.0,19785.0,HA,19690.0,Hawaiian Airlines Inc.,05375,D,HA,Hawaiian Airlines Inc.,3,3.0,13830,1383002,33830,OGG,"Kahului, HI",HI,15.0,Hawaii,US,United States,2,12173,1217305,32134,HNL,"Honolulu, HI",HI,15.0,Hawaii,US,United States,2,6,608,1,2018,4,10,1,F,DU,NaN,608,Boeing 717-200,6,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927,Domestic,Domestic,Domestic,HNL_OGG
349159,857.0,859.0,27193800.0,109952.0,81504.0,479954.0,0.0,100.0,36143.0,19001.0,HA,19690.0,Hawaiian Airlines Inc.,05375,D,HA,Hawaiian Airli

In [23]:
df_top_20_airlines_pivot = df_top_20_airlines.pivot_table(index = ['UNIQUE_CARRIER', 'Route_Type'], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).reset_index()
df_top_20_airlines_pivot = df_top_20_airlines_pivot.merge(df_top_20_airline_ranks[['UNIQUE_CARRIER', 'Rank']], on = 'UNIQUE_CARRIER')
df_top_20_airlines_pivot.sort_values(['Rank', 'Route_Type'], inplace = True)
df_top_20_airlines_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers"}, inplace = True)
df_top_20_airlines_pivot

,Airline,Route_Type,Passengers,Rank
0,WN,Domestic,160578652.0,1
1,WN,International,4560615.0,1
4,AA,Domestic,119081636.0,2
5,AA,International,27186303.0,2
2,DL,Domestic,119611221.0,3
3,DL,International,23437824.0,3
6,UA,Domestic,84532551.0,4
7,UA,International,25853375.0,4
8,B6,Domestic,33547937.0,5
9,B6,International,8087960.0,5


In [24]:
df_top_20_airlines_pivot.to_csv('top_20_airlines_by_passengers_2018.csv', index = False)
df_top_20_airlines_pivot.to_sql("top_20_airlines_by_passengers_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

40

# Creating a pivot table showing arriving passensger totals for each airline-airport pair:

In [25]:
df_airport_airline_pivot = df_flights.pivot_table(index = ['UNIQUE_CARRIER', 'DEST', "Route_Type"], values = 'PASSENGERS', aggfunc = 'sum').sort_values('PASSENGERS', ascending = False).query("PASSENGERS > 0").reset_index()
df_airport_airline_pivot.rename(columns={"UNIQUE_CARRIER":"Airline","DEST":"Airport","PASSENGERS":"Passengers"},inplace=True)
df_airport_airline_pivot

,Airline,Airport,Route_Type,Passengers
0,DL,ATL,Domestic,30464398.0
1,AA,DFW,Domestic,19283327.0
2,AA,CLT,Domestic,12510203.0
3,UA,ORD,Domestic,10421303.0
4,WN,MDW,Domestic,9834058.0
...,...,...,...,...
7926,24Q,SIR,International,1.0
7927,0CQ,PHL,Domestic,1.0
7928,24Q,ATL,International,1.0
7929,0CQ,ACK,Domestic,1.0


The output of the following two cells should match; otherwise, I've lost some passengers in the process of creating my pivot table.

In [26]:
df_airport_airline_pivot['Passengers'].sum()

951220029.0

In [27]:
df_flights['PASSENGERS'].sum()

951221969.0

In [28]:
df_airport_airline_pivot.to_csv('airport_airline_pairs_2018.csv', index = False)
df_airport_airline_pivot.to_sql('airline_airport_pairs_2018', if_exists = 'replace', con = elephantsql_engine, index = False)

931

# Creating a sample of the data that can be used for an interactive comparison chart:

In order to demonstrate how Plotly can be used to create a chart that lets users choose on which variables to compare data, I'll create a simplified version of the table that contains all of the top 4 airlines' passenger traffic at all of the top 5 airports. This table will also distinguish between domestic and international traffic.

In [29]:
top_5_airports = df_flights_domestic_destination_airports.pivot_table(index = "DEST", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).index[0:5].to_list()
top_5_airports

['ATL', 'LAX', 'ORD', 'DFW', 'JFK']

In [30]:
top_4_airlines = df_flights.pivot_table(index = "UNIQUE_CARRIER", values = "PASSENGERS", aggfunc = "sum").sort_values("PASSENGERS", ascending = False).index[0:4].to_list()
top_4_airlines

['WN', 'AA', 'DL', 'UA']

In [31]:
df_flights_excerpt_for_pivot_table = df_flights_domestic_destination_airports.query("UNIQUE_CARRIER in @top_4_airlines & DEST in @top_5_airports")
df_flights_excerpt_for_pivot_table

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP,CARRIER_GROUP_NEW,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50,Code_x,Plane_Type_Text,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,Origin_Region,Destination_Region,Route_Type,ORIGIN_DEST
3596,0.0,1.0,103900.0,273.0,186.0,0.0,0.0,731.0,137.0,112.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,627,1,2018,4,11,2,L,DU,NaN,627,Boeing 777-200ER/200LR/233LR,6,"Jet, 2-Engine",1,Passenger Configuration,ATL,33.640,-84.427,DFW,32.896,-97.037,Domestic,Domestic,Domestic,ATL_DFW
3597,0.0,1.0,50200.0,181.0,134.0,0.0,0.0,731.0,142.0,117.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,6,699,1,2018,4,11,2,L,DU,NaN,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,ATL,33.640,-84.427,DFW,32.896,-97.037,Domestic,Domestic,Domestic,ATL_DFW
3602,0.0,1.0,103900.0,273.0,176.0,0.0,0.0,731.0,107.0,90.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,627,1,2018,4,11,2,L,DU,NaN,627,Boeing 777-200ER/200LR/233LR,6,"Jet, 2-Engine",1,Passenger Configuration,DFW,32.896,-97.037,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_DFW
3603,0.0,1.0,50200.0,181.0,135.0,0.0,0.0,731.0,120.0,99.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48.0,Texas,US,United States,74,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,699,1,2018,4,11,2,L,DU,NaN,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,DFW,32.896,-97.037,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_DFW
3606,0.0,1.0,50200.0,181.0,161.0,0.0,0.0,594.0,130.0,104.0,AA,19805.0,American Airlines Inc.,0A050,D,AA,American Airlines Inc.,3,3.0,13303,1330303,32467,MIA,"Miami, FL",FL,12.0,Florida,US,United States,33,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,699,1,2018,4,11,2,L,DU,NaN,699,Airbus Industrie A321/Lr,6,"Jet, 2-Engine",1,Passenger Configuration,MIA,25.793,-80.291,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348976,412.0,411.0,20461760.0,81783.0,75597.0,93143.0,8766.0,404.0,37671.0,25620.0,DL,19790.0,Delta Air Lines Inc.,01260,D,DL,Delta Air Lines Inc.,3,3.0,13204,1320402,31454,MCO,"Orlando, FL",FL,12.0,Florida,US,United States,33,10397,1039707,30397,ATL,"Atlanta, GA",GA,13.0,Georgia,US,United States,34,6,622,1,2018,3,8,1,F,DU,NaN,622,Boeing 757-200,6,"Jet, 2-Engine",1,Passenger Configuration,MCO,28.429,-81.316,ATL,33.640,-84.427,Domestic,Domestic,Domestic,ATL_MCO
348983,419.0,417.0,16732610.0,668

The following pivot table contains rows for each airline/airport/route type pair (e.g. ATL-DL-International; DFW-AA-Domestic, etc.). Since there are 5 airports, 4 airlines, and 2 route types, the pivot table could have up to 40 rows. In practice, however, it will contain a smaller number, since not all of these airlines fly domestically and internationally from every airport.

In [32]:
df_t5_airports_t4_airlines_pivot = df_flights_excerpt_for_pivot_table.pivot_table(index = ["DEST", "UNIQUE_CARRIER", "Route_Type"], values = "PASSENGERS", aggfunc = "sum").reset_index().sort_values(['DEST', 'UNIQUE_CARRIER', 'Route_Type']).reset_index(drop=True)
df_t5_airports_t4_airlines_pivot.rename(columns={"DEST":"Airport","UNIQUE_CARRIER":"Airline","PASSENGERS":"Passengers"},inplace=True)
df_t5_airports_t4_airlines_pivot

,Airport,Airline,Route_Type,Passengers
0,ATL,AA,Domestic,1271729.0
1,ATL,AA,International,399.0
2,ATL,DL,Domestic,30464398.0
3,ATL,DL,International,4585330.0
4,ATL,UA,Domestic,566751.0
5,ATL,WN,Domestic,5003515.0
6,ATL,WN,International,89823.0
7,DFW,AA,Domestic,19283327.0
8,DFW,AA,International,2530917.0
9,DFW,DL,Domestic,815244.0


In [33]:
df_t5_airports_t4_airlines_pivot.to_csv('t5_airports_t4_airlines_2018.csv', index = False)
df_t5_airports_t4_airlines_pivot.to_sql("t5_airports_t4_airlines_2018",if_exists = 'replace', con = elephantsql_engine, index = False)

34